## 크롬 드라이버 프로세스 강제 종료

In [1]:
import psutil
import os
import signal

process_name = "chrome"  # 강제 종료하려는 프로세스 이름

for proc in psutil.process_iter(['pid', 'name']):
    try:
        if process_name.lower() in proc.info['name'].lower():
            pid = proc.info['pid']
            print(f"Forcefully killing process {proc.info['name']} with PID {pid}")
            # signal.SIGKILL: 프로세스를 강제로 종료하며, 운영 체제에서 이를 무시할 수 없습니다.
            os.kill(pid, signal.SIGKILL)  # SIGKILL 신호로 강제 종료
            print(f"Process {proc.info['name']} killed successfully")
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess) as e:
        print(f"Error killing process {proc.info['name']} (PID {proc.info['pid']}): {e}")


## 셀레니움 세팅

In [ ]:
## reference - https://thecho7.tistory.com/entry/Python과-Selenium을-활용한-웹-크롤링-실습-1

# 느낌표(!)는 Jupyter에서 패키지를 설치할 때 필요한 커맨드입니다
!pip install selenium

# 우리는 Chrome을 사용할건데, 이 패키지를 설치하면 버전 관리를 손쉽게 할 수 있습니다
# cmd + shift + p를 통해 reload windows를 하면 코드 경고 사라집니다
# https://github.com/SergeyPirogov/webdriver_manager
!pip install webdriver-manager

# 데이터 출력을 위한 라이브러리
!pip install pandas

## import

In [95]:
# 작업에 필요한 패키지를 불러옵니다
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
import re


# 데이터 출력
import pandas as pd

# Chrome 브라우저를 오픈합니다
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get("https://www.naver.com")

# 현재 시각 (한국 시간 기준)
current_time = datetime.now()


## 크롤링 최적화

In [97]:

## 드라이버 최적화
def driver_Settings():
    options = Options()

    # 웹드라이버 종료 안시키고 유지
    #options.add_experimental_option("detach", True)

    # 주석해제하면 헤드리스 모드
    options.add_argument("--headless")  # 헤드리스 모드로 실행

    # Windows 10 운영 체제에서 Chrome 브라우저를 사용하는 것처럼 보이는 사용자 에이전트가 설정
    options.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    # 언어 설정을 한국어로 지정. 이는 웹 페이지가 한국어로 표시
    options.add_argument('lang=ko_KR')

    # 브라우저 창의 크기를 지정. 여기서는 너비 430px, 높이 932px로 설정
    options.add_argument('--window-size=932,932')

    # GPU 가속을 비활성화. GPU 가속이 활성화되어 있으면, Chrome이 GPU를 사용하여 그래픽을 렌더링하려고 시도할 수 있기때문. 일부 환경에서는 GPU 가속이 문제를 일으킬 수 있으므로 이 옵션을 사용하여 비활성화
    options.add_argument('--disable-gpu')

    # 정보 표시줄을 비활성화. 정보 표시줄은 Chrome 브라우저 상단에 나타나는 알림이나 메시지를 의미. 이 옵션을 사용하여 이러한 알림이 나타나지 않도록 설정.
    options.add_argument('--disable-infobars')

    # 확장 프로그램을 비활성화. Chrome에서 확장 프로그램을 사용하지 않도록 설정
    options.add_argument('--disable-extensions')

    #  자동화된 기능을 비활성화. 이 옵션은 Chrome이 자동화된 환경에서 실행되는 것을 감지하는 것을 방지
    options.add_argument('--disable-blink-features=AutomationControlled')

    # 자동화를 비활성화. 이 옵션은 Chrome이 자동화 도구에 의해 제어되는 것으로 감지되는 것을 방지
    options.add_argument('--disable-automation')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    return driver

# 시간 정보를 계산하는 함수
def parse_time_info(time_info):
    """
    Parse time info like '6시간 전', '1주 전', or '2024. 10. 23.' and return the calculated datetime.
    """
    # 정규표현식 패턴
    time_patterns = {
        "시간 전": r"(\d+)\s*시간 전",
        "분 전": r"(\d+)\s*분 전",
        "일 전": r"(\d+)\s*일 전",
        "주 전": r"(\d+)\s*주 전",
        "개월 전": r"(\d+)\s*개월 전",
        "년 전": r"(\d+)\s*년 전",
        "날짜": r"(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})\.",
    }

    # 상대적 시간 계산
    for unit, pattern in time_patterns.items():
        match = re.match(pattern, time_info)
        if match:
            if unit == "시간 전":
                return current_time - timedelta(hours=int(match.group(1)))
            elif unit == "분 전":
                return current_time - timedelta(minutes=int(match.group(1)))
            elif unit == "일 전":
                return current_time - timedelta(days=int(match.group(1)))
            elif unit == "주 전":
                return current_time - timedelta(weeks=int(match.group(1)))
            elif unit == "개월 전":
                return current_time - timedelta(days=int(match.group(1)) * 30)
            elif unit == "년 전":
                return current_time - timedelta(days=int(match.group(1)) * 365)
            elif unit == "날짜":
                year, month, day = map(int, match.groups())
                return datetime(year, month, day)

    # 파싱 실패 시 None 반환
    return None


In [98]:
driver = driver_Settings()


In [112]:
# 날짜 지정 변수 추가(예: 오늘부터 7일 전까지 수집)
days_back = 0  # 오늘만 수집
current_time = datetime.now()  # 현재 시간

# days_back이 0일 때만 오늘 날짜를 사용하고, 다른 경우는 timedelta를 사용하여 범위를 설정
if days_back == 0:
    start_date = current_time.replace(hour=0, minute=0, second=0, microsecond=0)  # 오늘 자정
else:
    start_date = current_time - timedelta(days=days_back)

# 원하는 페이지 수
page_nums = 5
search = "추락"

# 데이터를 저장할 리스트
data = []

# 0, 10, 20, 30
# Google 검색 결과에서 데이터 수집
for page_num in range(0, page_nums * 10, 10):
    driver.get(f"https://www.google.com/search?q={search}&sca_esv=1f2f2e42c8b63453&tbm=nws&sxsrf=ADLYWIJiMdOV1vEJqiO9AKtDvtz5QmvpWA:1735375337578&ei=6blvZ473Itu3vr0P08HGyA0&start={page_num}&sa=N&ved=2ahUKEwjOj7H0iMqKAxXbm68BHdOgEdk4FBDy0wN6BAgEEAQ&biw=933&bih=973&dpr=1.8")
    posts = driver.find_elements(By.CSS_SELECTOR, "#rso > div > div > div")

    for post in posts:
        try:
            post_info = post.find_elements(By.CSS_SELECTOR, "div > div > a > div > div:nth-child(2) > div")
            company = post_info[0].text if len(post_info) > 0 else "Unknown"
            title = post_info[1].text if len(post_info) > 1 else "Unknown"
            content = post_info[2].text if len(post_info) > 2 else "Unknown"
            time_info = post_info[-1].text if len(post_info) > 0 else "Unknown"
            post_url = post.find_element(By.CSS_SELECTOR, "div > div > a").get_attribute("href")

            # 시간 계산
            calculated_time = parse_time_info(time_info)
            

            # 날짜 필터링: start_date와 비교하여 수집할 데이터인지 확인
            if calculated_time >= start_date and calculated_time <= current_time:
                # 데이터 저장
                data.append({
                    "Company": company,
                    "Title": title,
                    "Content": content,
                    "Link": post_url,
                    "Calculated_time": calculated_time,
            
                })
            
            # 결과 출력
            # print(f"URL: {post_url} | Original Time: {time_info} | Calculated Time: {calculated_time}")
            # print(f"URL: {post_url} | Calculated Time: {calculated_time}")

        except Exception as e:
            print(f"Error processing post: {e}")

# 데이터 정렬: calculated_time 기준 최신순 정렬
try:
    # 정렬 시 이미 datetime인 경우 그대로 사용
    data = sorted(data, key=lambda x: x["Calculated_time"] if isinstance(x["Calculated_time"], datetime) 
    else datetime.strptime(x["Calculated_time"], "%Y-%m-%d %H:%M:%S"), reverse=True)
except ValueError as e:
    print(f"Date parsing error: {e}")
except TypeError as e:
    print(f"Type error: {e}")


# 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(data, columns=["Company", "Title", "Content", "Link", "Calculated_time"])

# pandas 옵션 설정: 생략 없이 모든 행과 열을 출력
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)  # 출력 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 해제
display(df)



,Company,Title,Content,Link,Calculated_time
0,한겨레,[영상] 제주항공 참사 생존자 “한쪽 엔진서 연기 난 뒤 화재”,전남 무안공항 항공기 추락 사고의 생존자가 비행기 후미 쪽 엔진 쪽에서 연기가 나 화재가 발생했다고 밝혔다. 29일 전남도와 소방본부 등의 말을...,https://www.hani.co.kr/arti/area/honam/1175290.html,2024-12-29 15:28:48.280720
1,노컷뉴스,"MBC, 여객기 추락 사고에 결국 '방송연예대상' 결방 결정","MBC가 제주항공 여객기 추락 사고 속 '방송연예대상'을 결방하고 뉴스특보 편성을 결정했다. MBC는 ""긴급 뉴스특보 편성으로 인해,...",https://www.nocutnews.co.kr/news/6268733,2024-12-29 14:28:48.280720
2,연합뉴스,[무안 제주항공 참사] 국내 역대 항공기 사고는…괌 추락 이후 최악,(서울=연합뉴스) 권혜진 기자 = 무안공항에서 29일 발생한 제주항공 참사로 대형 인명피해로 이어진 역대 항공기 사고도 다시 관심을 받고 있다.,https://www.yna.co.kr/view/AKR20241229032900003,2024-12-29 14:28:48.280720
3,한겨레,아제르 추락 여객기 생존자 “외부서 2번 굉음…파편 뚫고 들어와”,아제르바이잔 여객기 추락 사고 생존자들이 추락 직전 외부에서 굉음이 나고 정체 모를 파편들이 기체를 뚫고 안으로 들어왔다고 증언했다.,https://www.hani.co.kr/arti/international/europe/1175311.html,2024-12-29 14:28:48.280720
4,YTN,전남 무안공항 여객기 추락...탑승객 96명 사망,"전남 무안공항 여객기 추락…탑승객 96명 사망소방청 ""사망…남성 47명·여성 48명·...",https://www.ytn.co.kr/_ln/0115_202412291422368728,2024-12-29 14:28:48.280720
5,경향신문,‘추락 여객기 제조사’ 보잉 “탑승객·유족 애도…제주항공 지원할 것”,"항공기 제조사 보잉이 제주항공 항공기 추락 사고와 관련해 애도의 뜻을 밝혔다. 29일 로이터통신에 따르면, 보잉은 이날 “우리는 제주항공과 2216편...",https://www.khan.co.kr/article/202412291452001,2024-12-29 14:28:48.280720
6,조선일보,외신도 무안 사고 긴급 타전… “1997년 대한항공기 괌 추락 이후 최악 사고”,외신도 무안 사고 긴급 타전 1997년 대한항공기 괌 추락 이후 최악 사고 미국 등 주요국 외신은 29일 전남 무안국제공항에서 발생한 제주항공 여객기...,https://www.chosun.com/international/international_general/2024/12/29/CPVXHZI5LVFQLMNEONFFAD4RH4/,2024-12-29 13:28:48.280720
7,한겨레,무안공항 181명 탄 제주항공 외벽 충돌…최소 28명 사망,"29일 오전 전남 무안군 무안공항에서 승객 175명을 태운 여객기가 추락해 불길이 솟아오르고 있다. 무안/연합뉴스, 독자 제공.",https://www.hani.co.kr/arti/area/area_general/1175274.html,2024-12-29 13:28:48.280720
8,의협신문,"전남 여객기 추락, 지역 의료계 팔 걷었다 ""의료지원 아끼지 않겠다""","전라남도 무안국제공항에서 여객기 추락 사고가 발생, 지역 의료계가 팔을 걷고 적극 의료지원에 나섰다. 희생자에 대한 애도의 메시지도 전했다.29일...",https://www.doctorsnews.co.kr/news/articleView.html?idxno=157922,2024-12-29 13:28:48.280720
9,경향신문,전남소방 “제주항공 추락 여객기 꼬리 부분만 남아…사망 85명·실종 94명”,전남 무안국제공항에서 탑승객 181명을 태운 여객기가 착륙 중 추락하는 사고가 나 85명의 사망자가 발생했다. 소방당국은 사고현장과 여객기 내부...,https://www.khan.co.kr/article/202412291326001,2024-12-29 13:28:48.280720
